<a href="https://colab.research.google.com/github/javiervasquezufm/ELEMENTS-OF-MACHINE-LEARNING/blob/master/Bootstrap.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Laboatorio 6

En este laboratorio usaremos bootstrap para hacer estimaciones de la media y desviación estándar de algún estadístico de un grupo de datos.

In [0]:
import numpy as np
import pandas as pd

import matplotlib
import matplotlib.pyplot as plt

import seaborn as sns

import statsmodels
import statsmodels.formula.api as smf

from sklearn.model_selection import KFold
from sklearn.utils import resample
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

print("Todos los paquetes han sido importados:")

Todos los paquetes han sido importados:


## Un ejemplo de bootstrap

Escribimos aquí lo que vimos en la presentación en clase: usamos la función resample de sklearn para realizar bootstrap sobre un grupo de datos



In [0]:
from sklearn.utils import resample

mean = [0,0]
cov = [[1.,0.25],[0.25,1.25]]

# Generamos observaciones (X,Y) a partir de una distribución aleatoria
np.random.seed(1)
X, Y = np.random.multivariate_normal(mean, cov, 100).T
X = X.reshape(-1, 1)
Y = Y.reshape(-1, 1)

alphas_hat = []

for _ in range(1000):
  X1, Y1 = resample(X, Y, replace=True, n_samples=100)
  [s_X, s_XY, _, s_Y] = np.cov(X1.T, Y1.T).reshape(-1, 1)
  alphas_hat.append(float((s_Y-s_XY)/(s_X+s_Y-2*s_XY)))

alpha_mean = np.mean(alphas_hat)
alpha_err = np.std(alphas_hat, ddof=1)
print(alpha_mean)

0.605932323049615


### Ejercicio 1

Basándote en el código anterior, haz una función bootstrap que tome como argumento un dataset y que regrese la media y desviación estándar del R^2 de una regresión lineal aplicada entre X y Y. Es decir, haz una función que usa bootstrap para encontrar un intervalo de confianza para el estadístico R^2 de una regresión lineal.

In [0]:
from sklearn.utils import resample

mean = [0,0]
cov = [[1.,0.25],[0.25,1.25]]

# Generamos observaciones (X,Y) a partir de una distribución aleatoria
np.random.seed(1)
X, Y = np.random.multivariate_normal(mean, cov, 100).T
X = X.reshape(-1, 1)
Y = Y.reshape(-1, 1)

alphas_hat = []
alphas_hat2 = []

for _ in range(1000):
  X1, Y1 = resample(X, Y, replace=True, n_samples=100)
  [s_X, s_XY, _, s_Y] = np.cov(X1.T, Y1.T).reshape(-1, 1)
  alphas_hat.append(float((s_X+  s_XY + _ + s_Y)/4))
  alphas_hat2.append(float((s_X+  s_XY + _ + s_Y)/4))
  
alpha_mean = np.mean(alphas_hat)
alpha_err = np.std(alphas_hat, ddof=1)
print(alpha_mean)

0.4797048105854568


In [0]:
from sklearn.utils import resample
# data sample
data = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6]
# prepare bootstrap sample
boot = resample(data, replace=True, n_samples=4, random_state=1)
print('Bootstrap Sample: %s' % boot)
# out of bag observations
oob = [x for x in data if x not in boot]
print('OOB Sample: %s' % oob)

Bootstrap Sample: [0.6, 0.4, 0.5, 0.1]
OOB Sample: [0.2, 0.3]


In [0]:
s_X+s_XY+_+s_Y/4

array([0.86558883])

## Ejercicio de aplicación

Utiliza los datos de cáncer de mama (los del laboratorio 3) y el procedimiento de bootstrap para hacer estimaciones de la media y desviación estándar de los coeficientes que arroja una regresión logística múltiple y compara estas estimaciones con las que da la regresión sobre la muestra original


In [0]:
from sklearn.datasets import load_breast_cancer
data = load_breast_cancer()
bc = pd.DataFrame(data.data, columns=data.feature_names) 
bc["Clase"] = pd.DataFrame(data.target, columns=['Clase']) 
est = smf.Logit(bc['Clase'], bc[bc.columns[0:5]]).fit()
est.summary2()



Optimization terminated successfully.
         Current function value: 0.150936
         Iterations 10


<class 'statsmodels.iolib.summary2.Summary'>
"""
                           Results: Logit
====================================================================
Model:                 Logit             No. Iterations:    10.0000 
Dependent Variable:    Clase             Pseudo R-squared:  0.771   
Date:                  2019-03-06 15:14  AIC:               181.7655
No. Observations:      569               BIC:               203.4849
Df Model:              4                 Log-Likelihood:    -85.883 
Df Residuals:          564               LL-Null:           -375.72 
Converged:             1.0000            Scale:             1.0000  
--------------------------------------------------------------------
                  Coef.   Std.Err.    z    P>|z|    [0.025   0.975] 
--------------------------------------------------------------------
mean radius        8.4813   1.2165  6.9719 0.0000    6.0970  10.8656
mean texture      -0.3442   0.0578 -5.9554 0.0000   -0.4574  -0.2309
mean perimeter    -0.7017   0.1718 -4.0834 0.0000   -1.0384  -0.3649
mean area         -0.0600   0.0075 -8.0505 0.0000   -0.0747  -0.0454
mean smoothness -108.0860  19.1643 -5.6400 0.0000 -145.6474 -70.5246
====================================================================

"""

In [0]:
values = bc.values
n_iterations = 1000
n_size = int(len(data) * 0.50)
stats = list()
for i in range(n_iterations):
	train = resample(values, n_samples=n_size)
	test = np.array([x for x in values if x.tolist() not in train.tolist()])
	model = DecisionTreeClassifier()
	model.fit(train[:,:-1], train[:,-1])
	predictions = model.predict(test[:,:-1])
	score = accuracy_score(test[:,-1], predictions)
	print(score)
	stats.append(score)
alpha = 0.95
p = ((1.0-alpha)/2.0) * 100
lower = max(0.0, np.percentile(stats, p))
p = (alpha+((1.0-alpha)/2.0)) * 100
upper = min(1.0, np.percentile(stats, p))
result = (alpha*100, lower*100, upper*100)
print('%.1f confidence interval %.1f%% and %.1f%%' % result)

0.6713780918727915
0.6254416961130742
0.872791519434629
0.8056537102473498
0.6254416961130742
0.6254416961130742
0.8833922261484098
0.46996466431095407
0.6254416961130742
0.9063604240282686
0.6254416961130742
0.5176678445229682
0.36925795053003535
0.7897526501766784
0.37809187279151946
0.6254416961130742
0.6978798586572438
0.36925795053003535
0.5459363957597173
0.6254416961130742
0.8480565371024735
0.7102473498233216
0.5477031802120141
0.7296819787985865
0.657243816254417
0.4134275618374558
0.6254416961130742
0.5265017667844523
0.8215547703180212
0.6254416961130742
0.6254416961130742
0.8533568904593639
0.5318021201413428
0.36925795053003535
0.8180212014134276
0.9169611307420494
0.8568904593639576
0.6254416961130742
0.8780918727915195
0.36925795053003535
0.9081272084805654
0.7544169611307421
0.8551236749116607
0.7084805653710248
0.7067137809187279
0.36925795053003535
0.8568904593639576
0.7897526501766784
0.6254416961130742
0.6749116607773852
0.7932862190812721
0.6254416961130742
0.62544